Here we want to find the fields that they agreed on, and therefore, were not discussed in the damn
meeting! 

We relabel everything that has mustard as double-crop.

In [1]:
import pandas as pd
import csv

import os, os.path
import sys
import shutil # to move files from one directory to another

In [2]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses_set_1.xlsx")
response_set_1_sheet_names = response_set_1_xl.sheet_names  # see all sheet names

In [4]:
print (choices_set_1_sheet_names)
print (response_set_1_sheet_names)

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6']
['S1 - F6', 'S1 - F5', 'S1 - F4', 'S1 - F3', 'S1 - F2', 'S1 - F1']


In [5]:
a_choice_sheet = choices_set_1_xl.parse(choices_set_1_sheet_names[3])
sample_response = response_set_1_xl.parse(response_set_1_sheet_names[3])

In [6]:
print (list(a_choice_sheet.columns))

['Question_in_set', 'Question_overall', 'ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude', 'longitude', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres', 'ExctAcr', 'LstSrvD', 'county']


## Count number of questions

In [7]:
question_count = 0

for a_choice_sheet in choices_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [294] questions.


# Disagreements start here:

In [8]:
## Define the damn output dataframe
unwanted_opinions = ["kirtir@wsu.edu"]

output_df = pd.DataFrame(columns=['Disagreement_form', 'disagreement_Question', \
                                  'opinion_count', 'ID', "PerrysVote", "AndrewsVote", "TimsVote"], 
                         index=range(question_count))
output_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google Disagreement_forms sheets
    # this is the Disagreement_form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # Disagreement_form sheet names of choices excel sheets
    choice_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choice_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    extended_choices = pd.concat([extended_choices, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if "QUESTION" in a_col_name:
            question_number = a_col_name.split()[1]
            
            output_df.loc[curr_row, "Disagreement_form"] = sheet_numeric_part
            output_df.loc[curr_row, "disagreement_Question"] = question_number
            output_df.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            output_df.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
             
            output_df.loc[curr_row, "PerrysVote"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            output_df.loc[curr_row, "AndrewsVote"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            output_df.loc[curr_row, "TimsVote"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1

extended_output = pd.merge(output_df, extended_choices, on=['ID'], how='left')
extended_output.head(2)

disagreement_table = extended_output[extended_output.opinion_count != 1].copy()
# disagreement_table = disagreement_table[disagreement_table.CropTyp != "grass hay"].copy()
disagreement_table.reset_index(inplace=True, drop=True)

disagreement_table.drop(labels=['TOA_RGB', 'latitude', 'longitude', 'Irrigtn', 'DataSrc', 'Acres',
                                'ExctAcr', 'LstSrvD', 'county', 'Question_in_set', 'Question_overall',
                                'corrected_RGB'], 
                         axis='columns', inplace=True)


disagreement_table.head(1)

,Disagreement_form,disagreement_Question,opinion_count,ID,PerrysVote,AndrewsVote,TimsVote,NDVI_TS_Name,CropTyp
0,6,8,3,107177_WSDA_SF_2018,Double Crop,Either double or mustard crop,Single Crop,FranklinYakima2018_107177_WSDA_SF_2018.png,grass seed


In [9]:
print (extended_output.shape)
extended_output.head(2)

(294, 21)


,Disagreement_form,disagreement_Question,opinion_count,ID,PerrysVote,AndrewsVote,TimsVote,Question_in_set,Question_overall,NDVI_TS_Name,...,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,6,1,1,157415_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,1,251,Walla2015_157415_WSDA_SF_2015.png,...,157415_WSDA_SF_2015_46.03408928_-118.6901155_T...,46.034089,-118.690116,alfalfa seed,sprinkler,wsda,28,27.675520,2015/06/04 00:00:00,Walla Walla
1,6,2,1,155601_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,2,252,Walla2015_155601_WSDA_SF_2015.png,...,155601_WSDA_SF_2015_46.0291209_-118.73977856_T...,46.029121,-118.739779,alfalfa seed,sprinkler,wsda,111,110.768754,2015/06/04 00:00:00,Walla Walla


In [10]:
drop_cols = ['Disagreement_form', 'disagreement_Question', 'NDVI_TS_Name', 'corrected_RGB', 
             'TOA_RGB', 'latitude', 'longitude', 'Irrigtn', 'DataSrc', 'Acres',
             'ExctAcr', 'LstSrvD', 'county', 'Question_in_set', 'Question_overall']
extended_output.drop(labels=drop_cols, axis=1, inplace=True)
extended_output.head(2)

,opinion_count,ID,PerrysVote,AndrewsVote,TimsVote,CropTyp
0,1,157415_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,alfalfa seed
1,1,155601_WSDA_SF_2015,Single Crop,Single Crop,Single Crop,alfalfa seed


In [11]:
pre_meeting_agreements = extended_output[extended_output.opinion_count==1].copy()
print (pre_meeting_agreements.shape)
double_place=['Mustard Crop', 'Either double or mustard crop']
#
# Correct Perry's vote so that mustard shit is labeled as double-cropped
#

pre_meeting_agreements['PerryVCorrected']=pre_meeting_agreements['PerrysVote']
idx = pre_meeting_agreements[pre_meeting_agreements.PerryVCorrected.isin(double_place)].index
pre_meeting_agreements.loc[idx, "PerryVCorrected"] = 'Double Crop'
pre_meeting_agreements.head(2)

#
# Correct Andrew's vote so that mustard shit is labeled as double-cropped
#
pre_meeting_agreements['AndrewVCorrected']=pre_meeting_agreements['AndrewsVote']
idx = pre_meeting_agreements[pre_meeting_agreements.AndrewVCorrected.isin(double_place)].index
pre_meeting_agreements.loc[idx, "AndrewVCorrected"] = 'Double Crop'
pre_meeting_agreements.head(2)

#
# Correct Tim's vote so that mustard shit is labeled as double-cropped
#

pre_meeting_agreements['TimsVCorrected']=pre_meeting_agreements['TimsVote']
idx = pre_meeting_agreements[pre_meeting_agreements.TimsVCorrected.isin(double_place)].index
pre_meeting_agreements.loc[idx, "TimsVCorrected"] = 'Double Crop'
print (pre_meeting_agreements.shape)
pre_meeting_agreements.head(2)



pre_meeting_agreements = pre_meeting_agreements[pre_meeting_agreements.PerryVCorrected==
                                                pre_meeting_agreements.AndrewVCorrected].copy()

pre_meeting_agreements = pre_meeting_agreements[pre_meeting_agreements.PerryVCorrected==
                                                pre_meeting_agreements.TimsVCorrected]

print (pre_meeting_agreements.shape)

# create Vote column in pre_meeting_agreements
pre_meeting_agreements['Vote'] = 1
pre_meeting_agreements.head(2)

# Change Vote to 2 for double cropped fields
double_index = pre_meeting_agreements[pre_meeting_agreements.PerryVCorrected=="Double Crop"].index
pre_meeting_agreements.loc[double_index, 'Vote']=2
pre_meeting_agreements = pre_meeting_agreements[["ID", 
                                                 "PerrysVote", "AndrewsVote", "TimsVote", 
                                                 "PerryVCorrected", "AndrewVCorrected", "TimsVCorrected",
                                                 "Vote"]]

out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
out_name = out_folder + "set1_premeeting_consensus.csv"
pre_meeting_agreements.to_csv(out_name, index = False)

(174, 6)
(174, 9)
(174, 9)


In [12]:
# pre_meeting_agreements = pre_meeting_agreements[pre_meeting_agreements.CropTyp != "grass hay"].copy()
drop_cols = ['Disagreement_form', 'disagreement_Question', 'NDVI_TS_Name']
disagreement_table.drop(labels=drop_cols, axis=1, inplace=True)
print (disagreement_table.shape)
disagreement_table.head(2)

(120, 6)


,opinion_count,ID,PerrysVote,AndrewsVote,TimsVote,CropTyp
0,3,107177_WSDA_SF_2018,Double Crop,Either double or mustard crop,Single Crop,grass seed
1,2,106626_WSDA_SF_2018,Either double or mustard crop,Single Crop,Single Crop,grass seed


In [13]:
174+120

294

# Read Experts Consensus after the meeting

In [14]:
import pandas as pd
expert_label_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
file_name =  "Expert_label_Consensus_Set1_Meeting.xlsx"
Labels_xls = pd.ExcelFile(expert_label_dir + file_name)
sheet_names = Labels_xls.sheet_names  # see all sheet names
Expert_label_Consensus = Labels_xls.parse("Expert_label_Consensus")
needed_cols = ["ID", "Vote"]
Expert_label_Consensus = Expert_label_Consensus[needed_cols]
Expert_label_Consensus.head(2)

,ID,Vote
0,160546_WSDA_SF_2015,1
1,99837_WSDA_SF_2017,2


# Some of the fucking fields in the meeting had consensus, but we needed to discuss them!!!

Toss them out of Expert_label_Consensus and save the new and true Expert_label_Consensus!

In [19]:
disagreement_IDs = list(disagreement_table.ID.unique())
Expert_label_Consensus = Expert_label_Consensus[Expert_label_Consensus.ID.isin(disagreement_IDs)]

In [24]:
Expert_label_Consensus.Vote.unique()

Expert_label_Consensus = Expert_label_Consensus[Expert_label_Consensus.Vote.isin([1, 2])]
Expert_label_Consensus.Vote = Expert_label_Consensus.Vote.astype(int)
print (Expert_label_Consensus.Vote.unique())

[2 1]


In [25]:
Expert_label_Consensus.shape

(111, 2)

In [26]:
out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
out_name = out_folder + "set1_postmeeting_consensus.csv"
Expert_label_Consensus.to_csv(out_name, index = False)